## Population Resource Consumption

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

# mapping to the modules that make the app
sys.path.insert(0, "../../az-function/")

import yaml, ssl, asyncio, pickle, os, ast

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

Much of the Population growth is handled in scripts inside the Azure Function

In [2]:
# mapping to the modules that make the app

from app.connectors import cmdb_graph
from app.functions import consumption
from app.objects import time

c = cmdb_graph.CosmosdbClient()


executing local windows deployment


In [3]:
params = yaml.safe_load(open(os.path.join(os.getenv("abspath"),"app/configurations/popgrowthconfig.yaml")))
syllables = pickle.load(open(os.path.join(os.getenv("abspath"),"app/creators/specs/syllables.p"), "rb"))
username = 'notebook'

Time is updated by the `time` function. It is always running.

In [4]:
t = time.Time(c)
t.get_current_UTU()
params['currentTime'] = t.params['currentTime']
t

< time at: 2024-06-02T15:46:19.932267+00:00 UTU:17943 >

In [5]:
params

{'pop_health_requirement': 0.7,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'currentTime': 17943}

For this notebook, setting the `pop_health_requirement` to an arbitrary value just to examine.

# Calculation which planets will consume

I've given up on trying to get a list of planets with the consumption type. I'm convinced that this could be an Azure Cosmos issue that I can't solve. 

```
g.V().has('label','planet').
    project('planetName', 'grouping').
        by('name').
        by(in('inhabits').out('isOf').as('species').groupCount().by('consumes'))
```

So first we are going to get a list of the locations who have populations. It could be any location, including planets. So I'm going to query based on 

In [14]:
consuming_planets = consumption.get_consuming_planets(c)
consuming_planets

[{'name': 'Damshamunling',
  'label': 'planet',
  'objid': '6135767546801',
  'consumes': [{'organics': 7}]},
 {'name': 'Teinspringsgio',
  'label': 'planet',
  'objid': '9482778660790',
  'consumes': [{'organics': 7}]}]

Then we will create an action event for each pop

In [17]:
message = consumption.get_consumption_message(consuming_planets[0])
message

{'agent': {'name': 'Damshamunling',
  'label': 'planet',
  'objid': '6135767546801',
  'consumes': [{'organics': 7}]},
 'action': 'consume'}

## Consuming



In [41]:
def reduce_location_resource(c,message, consumption):
    # find out if the location has the resource
    objid = message['agent']['objid']
    consuming = list(consumption.keys())[0]
    quantity = list(consumption.values())[0]
    resource_query = f"""
    g.V().has('objid','{objid}').out('has').has('label','resource').has('name','{consuming}').valuemap()
    """
    c.run_query(resource_query)
    if len(c.res) != 1:
        print(f"{objid} has a resource issue - c.res:{c.res}")
    resource = c.clean_nodes(c.res)[0]
    if resource['volume'] > quantity:
        new_volume = resource['volume'] - quantity
        patch_resource_query = f"""
        g.V().has('objid','{objid}').out('has').has('label','resource').has('name','{consuming}')
            .property('volume', {new_volume})
        """
        c.run_query(patch_resource_query)
        print(f"resources on {message['agent']['name']} reduced by {quantity}, {resource['volume']}-> {new_volume}")
    if resource['volume'] <= quantity:
        new_volume = 0
        patch_resource_query = f"""
        g.V().has('objid','{objid}').out('has').has('label','resource').has('name','{consuming}')
            .property('volume', {new_volume})
        """
        c.run_query(patch_resource_query)
        print(f"resources on {message['agent']['name']} reduced by {quantity}, People at this location will starve.")
    return resource


for resource in message['agent']['consumes']:
    # print(message['agent'], resource)
    resource =reduce_location_resource(c,message,resource)
    print(resource)
    

resources on Damshamunling reduced by 7, 979-> 972
{'volume': 979, 'name': 'organics', 'objid': '6439289323440', 'max_volume': 1007, 'description': 'bilogical material that can be consumed by pops', 'replenish_rate': 10, 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86', 'objtype': 'resource', 'id': '6439289323440'}


Grabbing that planets organics as well, and setting the amount of the organics to zero

Now that the resources are gone, the population should start to starve. 

In [51]:
consumption_df.loc[consumption_df['location_id']==planet['objid']].apply(lambda x: consumption.lower_health(c,params,x),axis=1)

0 pops will starve in nan


Series([], dtype: float64)

In [52]:
health_query =f"""
    g.V().has('objid','{planet['objid']}').as('location').in('inhabits')
        .haslabel('pop').as('pop')
        .out('isOf').as('species')
        .path()
            .by(valueMap('objid','name'))
            .by(valueMap('name','objid','health','username'))
            .by(valueMap('name','objid','consumes'))
"""

In [53]:
c.run_query(health_query)
out = c.res
pd.DataFrame([i['objects'][1] for i in out])

""


Run the following cells again and again to watch the health go down. 

In [32]:
consumption_df.loc[consumption_df['location_id']==planet['objid']].apply(lambda x: consumption.lower_health(c,params,x),axis=1)

7 pops will starve in 1425359096973


0    None
dtype: object

In [33]:
c.run_query(health_query)
out = c.res
pd.DataFrame([i['objects'][1] for i in out])

,name,objid,health,username
0,[Aythakorei Or],[9427021644478],[0.5999999999999999],[Billmanserver]
1,[Aythakorei Rezche],[8742217486310],[0.5999999999999999],[Billmanserver]
2,[Aythakorei Burghuatoucal],[2128380883047],[0.5999999999999999],[Billmanserver]
3,[Aythakorei Ton],[6992584860189],[0.5999999999999999],[Billmanserver]
4,[Borgantasga Nak],[7444602298651],[0.5999999999999999],[Billmanserver]
5,[Borgantasga Bin],[7852618490329],[0.5999999999999999],[Billmanserver]
6,[Aythakorei Ziburgdiadan],[3415888692185],[0.5999999999999999],[Billmanserver]


In [46]:
consumption_df.loc[consumption_df['location_id']==planet['objid']].apply(lambda x: consumption.lower_health(c,params,x),axis=1)
c.run_query(health_query)
out = c.res
pd.DataFrame([i['objects'][1] for i in out])

7 pops will starve in 1425359096973


""
